In [1]:
import numpy as np
import sympy as sp
from sympy.solvers import solve

x= sp.Symbol("x", positive=True)

r1c=0.342
p1c=8.87e-2
p2c=8.87e-2

r1c/=2
p1c/=2
p2c/=2

#r1c-x
#p1c+x
#p2c+x

#([p1c+x][p2c+x]/[r1c-x])=2.3e-2
#[p1c+x][p2c+x]=2.3e-2*r1c-2.3e-2*x

ans=solve((p1c+x)*(p2c+x)-(2.3e-2*r1c-2.3e-2*x),x)
print((p1c+x)*(p2c+x),",",(2.3e-2*r1c-2.3e-2*x))
print(ans)

r1c-=ans[0]
p1c+=ans[0]
p2c+=ans[0]

print("r1: ",r1c)
print("p1: ",p1c)
print("p2: ",p2c)

(x + 0.04435)**2 , -0.023*x + 0.003933
[0.0154612894288134]
r1:  0.155538710571187
p1:  0.0598112894288134
p2:  0.0598112894288134


In [2]:
#finds equilibrium concentrations given a list of reactants, products, coefficients of reactants, coefficients of products,
#and equilibrium constant k(c)
def FindEquilibrium(reactants, products, coRe, coPro, k, UseAllReactants=False):
    import numpy as np
    import sympy as sp
    from sympy import solveset, S, FiniteSet
    x= sp.Symbol("x", positive=True)
    lhs=1
    rhs=k
    for i in range(len(reactants)):
        rhs*=(reactants[i]-coRe[i]*x)**coRe[i]
    for i in range(len(products)):
        lhs*=(products[i]+coPro[i]*x)**coPro[i]
    if(not UseAllReactants):
        print(rhs,"=",lhs)
        ans=solveset(lhs-rhs,x, domain=S.Reals)
    else:
        ans=FiniteSet(min(reactants))
        print("UseAllReactants is set to True. Reaction will proceed until a reactant is consumed completely.")
    print("x=",ans)
    valid_x_list=[]
    react_out=[]
    prod_out=[]
    for i in range(len(ans)):
        broken=False
        for j in range(len(reactants)):
            if(reactants[j]-ans.args[i]<0):
                broken=True
                break
        for j in range(len(products)):
            if(products[j]+ans.args[i]<0):
                broken=True
                break
        if(not broken):
            valid_x_list.append(ans.args[i])
            print("\nusing valid x: ",ans.args[i])
            for j in range(len(reactants)):
                print("r"+str(j+1)+": ",reactants[j]-ans.args[i]*coRe[j])
                react_out.append(reactants[j]-ans.args[i]*coRe[j])
            for j in range(len(products)):
                print("p"+str(j+1)+": ",products[j]+ans.args[i]*coPro[j])
                prod_out.append(products[j]+ans.args[i]*coPro[j])
    return np.array(valid_x_list,dtype=np.float),np.array(react_out,dtype=np.float),np.array(prod_out,dtype=np.float)

#FindEquilibrium([0.218],[1.34e-2,1.93],[0],[1,1],1.8e-4)

In [3]:
#k2=exp(-dH/8.3145e-3*(1/T2-1/T1)+log(k1))
#np.exp(198/8.3145e-3*(1/1260-1/1150)+np.log(0.365))

def GetNewKConstant(k1,T1,T2,deltaH):
    return np.exp(-deltaH/8.3145e-3*(1/T2-1/T1)+np.log(k1))

#GetNewKConstant(322,350,453,-11.7)

In [4]:
#Finds pH given H+ concentration
def pH(Hydronium_Concentration):
    import numpy as np
    return -np.log10(Hydronium_Concentration)

#Finds H+ concentration given pH
def Hp(pH):
    import numpy as np
    return 10**(-pH)

#Finds pH using the Henderson-Hasselbalch equation. Provide either Ka or Kb, the concentrations and whether or not
#to get a more accurate ratio of products/reactants by using ICE tables with the Ka.
def HendersonHasselbalch(react_concentration, product_concentration, Ka=None, Kb=None, PerformIce=False):
    assert(Ka is not None or Kb is not None) #ensure an equilibrium constant exists
    import numpy as np
    if(not PerformIce):
        if(Ka):
            return -np.log10(Ka)+np.log10(product_concentration/react_concentration)
        else:
            return 14-(-np.log10(Kb)+np.log10(product_concentration/react_concentration))
    else:
        raise Exception('Ice tables not yet implemented.')

def pHChangeOfAcidInBuffer(molarityStrongAcid,molarityBufferAcid,molarityBufferBase,Ka):
    ph_i=HendersonHasselbalch(molarityBufferAcid,molarityBufferBase,Ka)
    print("initial ph :",ph_i)
    out=FindEquilibrium([molarityStrongAcid,molarityBufferBase],[molarityBufferAcid],[1,1],[1],1,UseAllReactants=True)
    product_con=out[1][1]
    react_con=out[2][0]
    ph_f=HendersonHasselbalch(react_con,product_con,Ka)
    print("final ph :",ph_f)
    dPh=ph_f-ph_i
    print("dPh :",dPh)
    return dPh

def BufferBuilder(TargetPh, Ka):
    import numpy as np
    ratio=10**(TargetPh+np.log10(Ka))
    print("moles reactant: ",1/ratio,"\nmoles product: ",ratio/ratio)
    return np.array([ratio/ratio,1/ratio],dtype=np.float)

BufferBuilder(9.85,4.8e-11)
#pHChangeOfAcidInBuffer(0.089,0.419,0.341,4.5e-4)
#HendersonHasselbalch(0.463, 0.22, Ka=4.8e-11)
#pH(3.01e-3)
#Hp(3.516)

moles reactant:  2.9427865512974045 
moles product:  1.0


array([1.        , 2.94278655])

In [11]:
#workspace
import numpy as np
import sympy as sp
from sympy.abc import x
from math import degrees, radians
from math import *

BufferBuilder()

initial ph : 9.496149282900015
UseAllReactants is set to True. Reaction will proceed until a reactant is consumed completely.
x= {0.114}

using valid x:  0.114000000000000
r1:  0
r2:  0.337000000000000
p1:  0.373000000000000
final ph : 9.211193574165957
dPh : -0.2849557087340582


14.284955708734058

In [ ]:
#finds pH using salt molarity and Ka or Kb of the salt. Make sure you give Ka of known substance
#if the acid is on LHS and Kb if base is on LHS
#eg. If acid Ka is known and it combines into some salt Conj. Acid+Conj. Base, input is Kb=1e-14/Ka
def FindPhOfSalt(molarity,Ka=None,Kb=None):
    import numpy as np
    pH=np.array([]);
    if(Ka):
        x_val,_,_=FindEquilibrium([molarity],[0,0],[1],[1,1],Ka)
        pH=-np.log10(np.array(x_val).astype(np.float))
    elif(Kb):
        x_val,_,_=FindEquilibrium([molarity],[0,0],[1],[1,1],Kb)
        pH=14+np.log10(np.array(x_val).astype(np.float))
    else:
        assert(Ka is not None and Kb is not None)
    print('pH is: ',pH)
    return pH
#FindPhOfSalt(0.217,Kb=1e-14/1.8e-5)